<a href="https://colab.research.google.com/github/kbrezinski/CS224W-GraphML/blob/main/notebooks/practice_deep_encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
print(torch.__version__)

from platform import python_version
print(python_version())

1.11.0+cu113
3.7.13


In [3]:
!pip install torch-scatter torch-sparse \
 torch-cluster torch-spline-conv torch-geometric \
-f https://data.pyg.org/whl/torch-1.11.0+cu113.html -q
#!pip install ogb

     |████████████████████████████████| 7.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 37.6 MB/s 
     |████████████████████████████████| 2.5 MB 35.0 MB/s 
     |████████████████████████████████| 750 kB 26.0 MB/s 
     |████████████████████████████████| 407 kB 4.7 MB/s 


In [20]:
import os
import numpy as np

from torch_geometric.data import Dataset, Data, Batch
from torch_geometric.loader import ClusterData, NeighborLoader, DataLoader

In [ ]:
# torch_geometric.data.batch.Batch 
# -creates batch of disconencted graphs from list

# torch_geometric.data.data.Data
# -creates single graph object

# torch_geometric.data.cluster.ClusterData/ClusterLoader
# -group nodes into smaller subgraphs and load them in batches for faster computation

# torch_geometric.data.sampler.NeighborSampler
# -samples specific number of nodes in neighbor
# -sample training nodes only using node_idx

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  after removing the cwd from sys.path.


In [6]:
g = Data(x=embed, edge_index=edges, edge_attr=edges_attr, y=ys)
g

Data(x=[100, 16], edge_index=[2, 500], edge_attr=[500], y=[100])

In [ ]:
# create batch of graphs using from_data_list
g2 = g
batch = Batch().from_data_list([g, g2])

In [8]:
# cluster data in 5 partitions, loop through N/5 nodes and retrieve edges
cluster = ClusterData(g, 5)
for c in cluster:
  print(c)
  break

Data(x=[20, 16], edge_attr=[500], y=[20], edge_index=[2, 47])


Computing METIS partitioning...
Done!


In [7]:
# for 4 nodes in the graph, sample (3 * 4), (10 * 40) neighbors for each iteration
sampler = NeighborLoader(g, num_neighbors=[3, 10],
                         batch_size=4, shuffle=False,
                         input_nodes=None) #data.train_mask,)
for s in sampler:
  print(s)
  break

Data(x=[49, 16], edge_index=[2, 65], edge_attr=[500], y=[49], batch_size=4)


In [24]:
# pre-transform - 
# tranform - 
# pre-filter - 

# torch_geometric.transforms - list of functions transformations for graphs

class SampleDataset(Dataset):
  def __init__(self, root, transform=None, pre_transform=None):
    super().__init__(root, transform, pre_transform)

    embed = torch.rand((100, 16), dtype=torch.float)  # 100 nodes, 16 features

    rows = np.random.choice(100, 500)
    cols = np.random.choice(100, 500)
    edges = torch.tensor([rows, cols])  # (2, 500) random edges
    edges_attr = np.random.choice(3, 500)  # 500 edges, choose from 0, 1 or 2
    ys = torch.rand((100)).round().long()

    # create data object
    self.data = Data(x=embed,              # [num_nodes, num_node_features]
                      edge_index=edges,     # [2, num_edges]
                      edge_attr=edges_attr, # [num_edges, num_edge_features]
                      y=ys,                 # [num_nodes]
                      )
    torch.save(self.data, os.path.join('data', f'data_graph.pt'))
    

  def len(self):
    return self.data.shape[0]

  def get(self):
    pass
    

dataset = SampleDataset(root='test')

In [25]:
dataloader = DataLoader(dataset, batch_size=1)

In [26]:
for i in dataloader:
  print(i)
  break

AttributeError: ignored

In [ ]:
# example transforms
# pre-transform - does so once its downloaded
# tranform - does so after its downloaded and retreived

transform = transforms.Compo se([
              transforms.AddTrainValTestMask('train_rest', num_val=500, num_test=500),
              transforms.TargetIndegree(),
])